# Защита персональных данных клиентов

 Необходимо разработать такой метод преобразования данных, чтобы по нему было сложно восстановить персональную информацию.


## Загрузка данных

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

$$
a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w'
$$

$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$
$$
(P^T (X^T X) P)^{-1} = P^{-1}(X^T X)^{-1} (P^T)^{-1}= P^{-1}(X^T X)^{-1} (P^{-1})^T
$$
$$
(XP)^Ty = P^T X^T y
$$
$$
w' = P^{-1}(X^T X)^{-1} (P^T)^{-1} P^T X^T y 
$$
$$
(P^T)^{-1} P^T = E 
$$

$$
w' = P^{-1}(X^T X)^{-1} X^T y
$$

In [ ]:
X = np.array([[1, 2, 5, 4 ], #обычная матрица признаков
            [1, 6, 7, 8 ],
            [1, 2, 8, 10],
            [1, 35, 8, 10],
            [1, 17, 38, 10]])
P = np.random.randint(1,100, size=(4,4))
 
y = np.array([5, 2, 1, 4, 1])
example = X.dot(P)
try:
    w = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), y)
    w1 = np.dot(np.dot(np.linalg.inv(np.dot(example.T, example)), example.T), y)
except:
    print('Ошибка')
a = X @ w
print(a)
a1 = example @ w1
print(a1)

[4.83677419 2.48967742 0.73290323 3.94064516 1.        ]
[4.83677419 2.48967742 0.73290323 3.94064516 1.        ]


<div class="alert alert-info">Все работает!
</div>

## Алгоритм преобразования

**Алгоритм**

...

In [ ]:
random_matrix = np.random.randint(1,100, size =(4,4)) #создадим случайную матрицу
print(random_matrix)
inv_matrix = np.linalg.inv(random_matrix)
print(inv_matrix)
print(np.linalg.det(random_matrix)) #посмотрим, является ли она обратимой, то есть посмотрим чему равен определитель

[[88  6 90 58]
 [77 45 25 24]
 [13  4 88 31]
 [75 58 63 30]]
[[ 0.08728611 -0.25289598 -0.15929519  0.19816868]
 [-0.07504326  0.20166149  0.11946839 -0.13969622]
 [ 0.06255608 -0.21974637 -0.11319976  0.17182844]
 [-0.20449939  0.70382847  0.40498526 -0.55284872]]
-499312.0000000016


<div class="alert alert-info">случайная матрица обратимая, поэтому можем приступать к проверке
</div>

## Проверка алгоритма

<div class="alert alert-info">проверим как работает r2 до и после предобразования
</div>

In [ ]:
features = df.drop('Страховые выплаты', axis=1) #до преобразования
target = df['Страховые выплаты']

class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T @ X) @ X.T @ y
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0
    
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.42494550286668


<div class="alert alert-info">значение r2 не отрицательное, не равно нулю и не больше единицы, значит, линейная регрессия работает
</div>

In [ ]:
features_1 = df.drop('Страховые выплаты', axis=1) #линейная регрессия из sklearn До преобразования
target_1 = df['Страховые выплаты']

features_train, features_valid, target_train, target_valid = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345) # отделите 25% данных для валидационной выборки

model = LinearRegression() # инициализируйте модель LinearRegression
model.fit(features_train, target_train) # обучите модель на тренировочной выборке
predictions_valid = model.predict(features_valid) # получите предсказания модели на валидационной выборке

print(r2_score(target_valid, predictions_valid))

0.4352275712702667


In [ ]:
features_new = features.values @ random_matrix #с применением алгоритма
target_new = target

class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T @ X) @ X.T @ y
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0
    
model = LinearRegression()
model.fit(features_new, target_new)
predictions = model.predict(features_new)
print(r2_score(target_new, predictions))

0.4249455024727228


In [ ]:
features_new_1 = features_1.values @ random_matrix #с применением алгоритма
target_new_1 = target_1

features_train, features_valid, target_train, target_valid = train_test_split(
    features_new_1, target_new_1, test_size=0.25, random_state=12345) # отделите 25% данных для валидационной выборки

model = LinearRegression() # инициализируйте модель LinearRegression
model.fit(features_train, target_train) # обучите модель на тренировочной выборке
predictions_valid = model.predict(features_valid) # получите предсказания модели на валидационной выборке

print(r2_score(target_valid, predictions_valid))

0.4352260213131829


<div class="alert alert-info">Как видно из моделей, до и после преобразования с помощью алгоритма (умножения исходной матрицы признаков на случайную матрицу), r2 показывает одинаковые результаты.
</div>